In [0]:
import numpy as np
import torch

## Direct Leniar regression

In [0]:
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [102, 119]], dtype='float32')

In [0]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### Create the tensors

In [4]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [5]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [102., 119.]])

In [6]:
w = torch.randn(2,3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.7819,  0.8167,  0.5277],
        [ 1.0301,  0.9372, -0.7775]], requires_grad=True)
tensor([-0.5258,  0.9185], requires_grad=True)


### The linear regression formula would be X * W.t + b

In [0]:
def model(x): 
  return x @ w.t() + b #@ operation is matrix multiplication in pytorch

In [8]:
# generate preds
preds = model(inputs)
preds

tensor([[133.9656, 105.4720],
        [176.2729, 127.3663],
        [207.5480, 171.0237],
        [133.8739, 117.5154],
        [168.7706, 107.5377]], grad_fn=<AddBackward0>)

In [9]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [102., 119.]])

The predictions are way-off cuz the weights are intialized randomly and need to be tuned

---
Lets start training the model

In [0]:
def mae_loss(p, t):
  diff = p - t
  return torch.sum(diff*diff)/diff.numel()

In [11]:
# compute the loss
loss = mae_loss(preds, targets)
print(loss)
loss.backward()

tensor(4998.3765, grad_fn=<DivBackward0>)


This means that each element is off by sqrt(40138.4219)

In [0]:
# back prop
learning_rate = 1e-5
with torch.no_grad():
  w -= w.grad * learning_rate
  b -= b.grad * learning_rate
  w.grad.zero_()
  b.grad.zero_()

In [14]:
new_preds = model(inputs)
loss = mae_loss(new_preds, targets)
print(loss)

tensor(3614.8953, grad_fn=<DivBackward0>)


## Encapsulating everything in a loop

In [18]:
epochs = 1000
for i in range(epochs):
  preds = model(inputs)
  loss = mae_loss(preds, targets)
  if not i % 100:
    print(f"Loss at EPOCH {i}: {loss}")
  loss.backward()
  learning_rate = 1e-5
  with torch.no_grad():
    w -= w.grad * learning_rate
    b -= b.grad * learning_rate
    w.grad.zero_()
    b.grad.zero_()

Loss at EPOCH 0: 21.998445510864258
Loss at EPOCH 100: 18.30776596069336
Loss at EPOCH 200: 15.246025085449219
Loss at EPOCH 300: 12.70604133605957
Loss at EPOCH 400: 10.598787307739258
Loss at EPOCH 500: 8.850568771362305
Loss at EPOCH 600: 7.400201320648193
Loss at EPOCH 700: 6.1969475746154785
Loss at EPOCH 800: 5.198680400848389
Loss at EPOCH 900: 4.370497226715088


In [19]:
final_preds = model(inputs)
print(final_preds)
print(targets)

tensor([[ 57.0187,  70.6763],
        [ 81.5241,  98.7803],
        [119.4759, 136.6692],
        [ 21.4886,  38.0425],
        [100.4622, 115.4315]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [102., 119.]])


#### There you go, Now lets use the pytorch built in functions to make this easier for us and explore batching in pytorch

In [0]:
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [102, 119],
                    [56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [102, 119],
                    [56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [102, 119]], dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy (targets)

In [0]:
from torch.utils.data import TensorDataset
train_ds = TensorDataset(inputs, targets) # coverts data in tensor tuples

In [0]:
# using dataloader to batch and shuffle data
from torch.utils.data import DataLoader
train_dl = DataLoader(train_ds, batch_size=5, shuffle=True)

## Torch.nn

In [0]:
from torch import nn

In [0]:
model = nn.Linear(3, 2)

In [38]:
# available methods of the model class: 
print(model.weight)
print(model.bias)
print(model.parameters()) # is a generator object

list(model.parameters())

Parameter containing:
tensor([[ 0.1933, -0.4874, -0.3710],
        [-0.1810, -0.5771, -0.0844]], requires_grad=True)
Parameter containing:
tensor([-0.3585,  0.1341], requires_grad=True)
<generator object Module.parameters at 0x7fb5449f14c0>


[Parameter containing:
 tensor([[ 0.1933, -0.4874, -0.3710],
         [-0.1810, -0.5771, -0.0844]], requires_grad=True),
 Parameter containing:
 tensor([-0.3585,  0.1341], requires_grad=True)]

In [39]:
# generating predictions 
preds = model(inputs)
preds

tensor([[-34.8532, -55.3726],
        [-49.3990, -72.5214],
        [-70.3661, -97.8372],
        [-15.3241, -46.2659],
        [-59.7767, -73.6619],
        [-34.8532, -55.3726],
        [-49.3990, -72.5214],
        [-70.3661, -97.8372],
        [-15.3241, -46.2659],
        [-59.7767, -73.6619],
        [-34.8532, -55.3726],
        [-49.3990, -72.5214],
        [-70.3661, -97.8372],
        [-15.3241, -46.2659],
        [-59.7767, -73.6619]], grad_fn=<AddmmBackward>)

In [0]:
import torch.nn.functional as F
# loss function, using built in loss functions
loss_fn = F.mse_loss

In [42]:
loss = loss_fn(preds, targets)
print(f"Loss is {loss}")

Loss is 23184.8125


In [0]:
# optimizer
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [0]:
def fit(num_epochs, model, loss_fn, opt):

  for epoch in range(num_epochs):
    for x_batch, y_batch in train_dl:
      preds = model(x_batch)
      loss = loss_fn(preds, y_batch)
      loss.backward()
      opt.step()
      opt.zero_grad()
    
    if not epoch % 100:
      print(f"Epoch [{epoch+1}/{num_epochs}] --> Loss: {loss}")

In [48]:
fit(1000, model, loss_fn, opt)

Epoch [1/1000] --> Loss: 67.52508544921875
Epoch [101/1000] --> Loss: 8.734457015991211
Epoch [201/1000] --> Loss: 6.698681831359863
Epoch [301/1000] --> Loss: 2.5612761974334717
Epoch [401/1000] --> Loss: 1.782958745956421
Epoch [501/1000] --> Loss: 1.7657442092895508
Epoch [601/1000] --> Loss: 0.6849534511566162
Epoch [701/1000] --> Loss: 0.4028114378452301
Epoch [801/1000] --> Loss: 0.5364033579826355
Epoch [901/1000] --> Loss: 0.33825045824050903
